### 4. EDA & 모델링(2)
- 위도, 경도를 이용한 시각화
- 이상치 제거

### 1. 데이터 읽어오기

In [ ]:
import pandas as pd

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')      
submission = pd.read_csv('sample_submission.csv')

train.head(3)

### 2. 데이터 확인

In [ ]:
# 파이썬 warning 무시
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# 시각화를 위한 라이브러리
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# 한글 폰트를 사용하기 위한 코드
fe = fm.FontEntry(fname = 'NotoSansKR-Regular.otf', name = 'NotoSansKR')
fm.fontManager.ttflist.insert(0, fe)
plt.rc('font', family='NotoSansKR')

## 코드 작성   
ax = train['next_arrive_time'].plot(figsize=(20,10), alpha=0.4)

ax.set_title('버스 운행시간')
ax.set_xlabel('인덱스')
ax.set_ylabel('운행시간')

plt.show()

In [ ]:
ax = train['next_arrive_time'].plot(figsize=(20,10), alpha=0.4)    

ax.set_title('버스 운행시간')    
ax.set_xlabel('인덱스')    
ax.set_ylabel('운행시간')    

ax.hlines(y = 20, xmin=0, xmax=len(train), colors='red')    
ax.hlines(y = 250, xmin=0, xmax=len(train), colors='red')    
ax.hlines(y = 600, xmin=0, xmax=len(train), colors='blue')    
ax.hlines(y = 2300, xmin=0, xmax=len(train), colors='green')    

plt.show()    

In [ ]:
a = train[train['next_arrive_time']<20]
b = train[(train['next_arrive_time']>=20) & (train['next_arrive_time']<250)]
c = train[(train['next_arrive_time']>=250) & (train['next_arrive_time']<600)]
d = train[(train['next_arrive_time']>=600) & (train['next_arrive_time']<2300)]
e = train[(train['next_arrive_time']>=2300)]

In [ ]:
x = ['20미만', '20~250', '250~600', '600~2300', '2300이상']
y = [len(a),len(b),len(c),len(d),len(e)]

# fig와 ax 객체 생성
fig, ax = plt.subplots(dpi=150)

ax.bar(x,y)

ax.set_title('버스 운행시간 구간 별 횟수')
ax.set_xlabel('버스 운행시간 구간')
ax.set_ylabel('횟수')

plt.show()

In [ ]:
label = ['b구간', '나머지']
value = [len(b)/len(train), (1-len(b)/len(train))]

plt.figure(dpi=150)

plt.title('버스 운행시간 구간 비율')

plt.pie(x = value, labels = label)
plt.legend()
plt.show()

In [ ]:
fig, ax = plt.subplots(dpi=150)

ax.hist(train[train['next_arrive_time']<600]['next_arrive_time'], bins=20)

ax.set_title('버스 운행시간 히스토그램')
ax.set_xlabel('운행시간')
ax.set_ylabel('횟수')

plt.show()

In [ ]:
x = ['운행시간<20', '운행시간>=20']
y = [a['distance'].mean(), pd.concat([b,c,d,e])['distance'].mean()]

plt.figure(dpi=150)

plt.title('운행시간 vs 거리')
plt.xlabel('운행시간')
plt.ylabel('거리')

plt.bar(x,y)
plt.show()

In [ ]:
x = ['20~250', '250~600']
y = [b['distance'].mean(), c['distance'].mean()]

plt.figure(dpi=150)

plt.title('운행시간 vs 거리')
plt.xlabel('운행시간')
plt.ylabel('거리')

plt.bar(x,y)
plt.show()

In [ ]:
x = train['distance']
y = train['next_arrive_time']

plt.figure(dpi = 150)

plt.title('거리 vs 운행시간')
plt.xlabel('거리')
plt.ylabel('운행시간')

plt.scatter(x,y, alpha = 0.3)

plt.show()

In [ ]:
import matplotlib.patches as patches

x = train['distance']
y = train['next_arrive_time']

plt.figure(dpi = 150)

plt.title('거리 vs 운행시간')
plt.xlabel('거리')
plt.ylabel('운행시간')

plt.scatter(x,y, alpha = 0.3)

# 이상치 표시하기 - 직사각형
plt.gca().add_patch(
    patches.Rectangle(
        (0, 1000),
        1000, 2000,               
        edgecolor = 'deeppink',
        fill=False,
    ))

plt.show()

In [ ]:
import folium

# 운행 거리가 1000 이하이면서 운행 시간이 1000 이상인 데이터의 좌표 정보 (list)
locations = train[(train['distance'] <= 1000) & (train['next_arrive_time'] >= 1000)][['next_latitude', 'next_longitude']].values.tolist()

# 지도 중심 좌표 - 위도와 경도의 평균
lat_mean = sum([lat[0] for lat in locations]) / len(locations)
long_mean = sum([long[1] for long in locations]) / len(locations)
center = [lat_mean, long_mean]

# 지도 그리기  
m = folium.Map(location=center, zoom_start=9, tiles='cartodbpositron')

# 점(Circle) 표시하기
for location in locations:
    folium.Circle(
        radius=300,
        location=location,
        fill=True
    ).add_to(m)

m

In [ ]:
temp = train[(train['distance'] > 1000) & (train['next_arrive_time'] < 1000)]

x = temp['distance']
y = temp['next_arrive_time']

plt.figure(dpi = 150)
plt.scatter(x,y, alpha = 0.3)

# 운행시간 분포 표시하기 - 타원
plt.gca().add_patch(
   patches.Ellipse(
      (4000, 600),
      8000, 800,               
      edgecolor = 'deeppink',
      fill=False,
      angle=3
   ))

plt.show()

In [ ]:
# 운행 시간이 2300 이상인 데이터의 좌표 정보 (list)
locations = train[train['next_arrive_time'] >= 2300][['next_latitude', 'next_longitude']].values.tolist()

# 지도 중심 좌표 - 위도와 경도의 평균
lat = sum([lat[0] for lat in locations]) / len(locations)
long = sum([long[1] for long in locations]) / len(locations)
center = [lat, long]

# 지도 그리기
m = folium.Map(location=center, zoom_start=12)

# 점(Circle) 표시하기
for location in locations:
    folium.Circle(
        radius=300,
        location=location,
        fill=True
    ).add_to(m)

m

In [ ]:
train[(train['next_station'] == '제주대학교') & (train['next_arrive_time'] >= 2300)]

In [ ]:
# 운행 시간이 긴 데이터
X = train[train['next_longitude'] == 126.561659]['next_arrive_time'].reset_index(drop=True).index
Y = train[train['next_longitude'] == 126.561659]['next_arrive_time'].reset_index(drop=True).values

plt.plot(X,Y)
plt.show()

### 3. 데이터 전처리

In [ ]:
my_list = list(train['route_nm'].unique()) + list(train['now_station'].unique()) + list(train['next_station'].unique())
my_dict = {text : i for i, text in enumerate(my_list)}
my_dict

In [ ]:
train[['route_nm', 'now_station', 'next_station']] = train[['route_nm', 'now_station', 'next_station']].applymap(lambda x : my_dict[x])
test[['route_nm', 'now_station', 'next_station']] = test[['route_nm', 'now_station', 'next_station']].applymap(lambda x : my_dict[x])

def text_to_time(text:str):
    return int(text[:2])

train['now_arrive_time'] = train['now_arrive_time'].map(text_to_time)
test['now_arrive_time'] = test['now_arrive_time'].map(text_to_time)

train.head()

In [ ]:
train = train[train['next_arrive_time'] < 1000]

In [ ]:
features = train.columns[2:-1]     
target = train.columns[-1]    

X = train[features]     
y = train[target]

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X,y, test_size=0.3, shuffle=False)

#데이터 shape 확인
print(f"X_train.shape : {X_train.shape}")
print(f"y_train.shape : {y_train.shape}")
print(f"X_valid.shape : {X_valid.shape}")
print(f"y_valid.shape : {y_valid.shape}")

### 4. 데이터 학습

In [ ]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators = 20)

In [ ]:
model.fit(X_train,y_train)

In [ ]:
# 학습된 모델을 이용해 결괏값 예측 후 상위 10개의 값 확인
predict = model.predict(test[features])
print('----------------------예측된 데이터의 상위 10개의 값 확인--------------------\n')
print(predict[:10])

### 5. CSV파일로 저장

In [ ]:
submission['next_arrive_time'] = predict     
submission.head()

In [ ]:
submission.to_csv('submission.csv', index=False, quoting=2, encoding="utf-8-sig", na_rep=0, float_format='%.6f')